<a href="https://colab.research.google.com/github/SaguPandya96/Research-Project-Fall-2020/blob/master/Ensemble_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import string
import random as rr
from textblob.classifiers import NaiveBayesClassifier as NBC
from textblob.classifiers import DecisionTreeClassifier
from sklearn.metrics import f1_score
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.utils import shuffle
import numpy as np

from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.metrics import f1_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
#load data
DATA_PATH1 = '/content/drive/My Drive/Research Project/chromium.csv'
data1 = pd.read_csv(DATA_PATH1, sep='\t')
data_top1 = data1.head()
print("Chromium Data")
data_top1

Chromium Data


,title,description,type
0,Define a way to enforce performance constraint...,\nThere are cases where tests should fail if a...,Bug
1,Websites that don't work with Chrome Browser,\nChrome Version : 3.0.195.10\r\nURLs (i...,Compat
2,Redraw issue/truncation with confirmation item...,\nChromium 0.3.155.0 (Developer Build 3546)\r\...,Bug
3,Implement CanvasRenderingContext2D for access ...,\nApplications that do background image proces...,Feature
4,ath9k: panic in ath_tx_last_beacon + 0x29a,\nToT (3.0 kernel) panic'd on resume. Was ass...,Bug


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
DATA_PATH2 = '/content/drive/My Drive/Research Project/linux_bugs_usage_ready.csv'
data2 = pd.read_csv(DATA_PATH2, sep='\t')
data_top2 = data2.head()
print("Linux Data")
data_top2

Linux Data


,title,message,importance,product,component
0,oops when using ide-cd with 2.5.45 and cdrecord,Please enter Exact Kernel version:2.5.45 Distr...,P2 normal,IO/Storage,IDE
1,NUMA-Q hangs during TSC initialization on boot.,Exact Kernel version: 2.5.46 Distribution: deb...,P2 normal,Platform Specific/Hardware,i386
2,Enabling shared pagetables causes KDE to wierd...,Exact Kernel version: 2.5.46-mm1 Distribution:...,P2 normal,Memory Management,Other
3,Dcache spirals out of control on 2.5.43-mm2,Exact Kernel version: 2.5.43-mm2 Distribution:...,P2 blocking,IO/Storage,Other
4,64GB highmem BUG(),Exact Kernel version: 2.5.40 Hardware Environm...,P2 normal,Memory Management,Other


In [6]:
#merge title and message
def merge_title_and_message(data, message_col_name='message'):
  data['text'] = data['title'] + ' ' + data[message_col_name]
  data = data.drop(['title'], axis=1)
  data = data.drop([message_col_name], axis=1)
  return data
data1 = merge_title_and_message(data1, message_col_name='description')
data_top1 = data1.head()
print("Chromium Merge Title and Message Data")
data_top1

Chromium Merge Title and Message Data


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that don't work with Chrome Browser \...
2,Bug,Redraw issue/truncation with confirmation item...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k: panic in ath_tx_last_beacon + 0x29a \nT...


In [7]:
data2 = merge_title_and_message(data2)
data_top2 = data2.head()
print("Linux Merge Title and Message Data")
data_top2

Linux Merge Title and Message Data


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using ide-cd with 2.5.45 and cdrecor...
1,P2 normal,Platform Specific/Hardware,i386,NUMA-Q hangs during TSC initialization on boot...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2.5.43-mm2 Ex...
4,P2 normal,Memory Management,Other,64GB highmem BUG() Exact Kernel version: 2.5.4...


In [8]:
def strip_punctuations(data, column_name='text'):
  '''
  Strips punctuations from the end of each token.
  This uses suggestion from https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
  to accomplish this really fast.
  '''
  translator = str.maketrans('', '', string.punctuation)
  data['text'] = data['text'].map(lambda s : str(s).translate(translator))
  return data

In [9]:
data1 = strip_punctuations(data1)
data_top1 = data1.head()
print("Chromium strip punctuations")
data_top1

Chromium strip punctuations


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that dont work with Chrome Browser nC...
2,Bug,Redraw issuetruncation with confirmation item ...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a nToT 30 ...


In [10]:
data2 = strip_punctuations(data2)
data_top2 = data2.head()
print("Linux strip punctuations")
data_top2

Linux strip punctuations


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2543mm2 Exact...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


In [11]:
def remove_linux_garbage(data):
  '''
  Linux data contains lots of garbage, e.g. memory addresses - 0000f800
  '''
  def is_garbage(w):
    return len(w) >= 7 and sum(c.isdigit() for c in w) >= 2

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if not is_garbage(w) else ' ', s.split())))
  return data
data = remove_linux_garbage(data2)
data_top2 = data2.head()
print("Remove linux garbage")
data_top2

Remove linux garbage


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on Exact Kerne...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


In [12]:
def cast_to_lowercase(data):
  data['text'] = data['text'].map(lambda s : s.lower())
  return data
data1 = cast_to_lowercase(data1)
data2 = cast_to_lowercase(data2)
data_top1 = data1.head()
print("Chromium Lowercase")
data_top1

Chromium Lowercase


,type,text
0,Bug,define a way to enforce performance constraint...
1,Compat,websites that dont work with chrome browser nc...
2,Bug,redraw issuetruncation with confirmation item ...
3,Feature,implement canvasrenderingcontext2d for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a ntot 30 ...


In [13]:
data_top2 = data2.head()
print("Linux Lowercase")
data_top2

Linux Lowercase


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord p...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs during tsc initialization on boot ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde to wierd...
3,P2 blocking,IO/Storage,Other,dcache spirals out of control on exact kerne...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [14]:
def remove_stopwords(data):
  stop_words = stopwords.words('english')
  translator = str.maketrans('', '', string.punctuation)
  stop_words = set([w.translate(translator) for w in stop_words]) # Apostrophes were removed already

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if w not in stop_words else ' ', s.split())))
  return data
data1 = remove_stopwords(data1)
data2 = remove_stopwords(data2)
data_top1 = data1.head()
print("Chromium Stopword")
data_top1

Chromium Stopword


,type,text
0,Bug,define way enforce performance constraint ...
1,Compat,websites work chrome browser nchrome ver...
2,Bug,redraw issuetruncation confirmation item d...
3,Feature,implement canvasrenderingcontext2d access ...
4,Bug,ath9k panic athtxlastbeacon 0x29a ntot 30 ke...


In [15]:
data_top2 = data2.head()
print("Linux Stopword")
data_top2

Linux Stopword


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please en...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ...
3,P2 blocking,IO/Storage,Other,dcache spirals control exact kernel vers...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [16]:
def remove_rare_words(data, min_count=3):
  wc = {} # WordCount
  def proc_word(s):
    for w in set(s.split()):
      if w in wc:
        wc[w] += 1
      else:
        wc[w] = 1

  for index, row in data.iterrows():
    proc_word(row['text'])

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if wc[w] >= min_count else ' ', s.split())))
  return data
data1 = remove_rare_words(data1)
data2 = remove_rare_words(data2)
data_top1 = data1.head()
print("Chromium rare words")
data_top1

Chromium rare words


,type,text
0,Bug,define way enforce performance constraint spec...
1,Compat,websites work chrome browser nchrome version ...
2,Bug,redraw confirmation item download bar nchrom...
3,Feature,implement canvasrenderingcontext2d access work...
4,Bug,ath9k panic ntot 30 kernel resume associ...


In [17]:
data_top2 = data2.head()
print("Linux rare words")
data_top2

Linux rare words


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please enter ex...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact kern...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ex...
3,P2 blocking,IO/Storage,Other,dcache control exact kernel version distribu...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [19]:
print("Ensemble Chromium Classification")
from sklearn.ensemble import RandomForestClassifier
def essemble_classify(data,extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'type' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)
  model = RandomForestClassifier(max_depth=50, random_state=0)
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 

  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')))

essemble_classify(data1)

Ensemble Chromium Classification
Vectors
legacy webaudio   remove nthis tracking bug issues legacy webaudio methods attributes need removed support prefixed audiocontext removedn
gn need mechanism share parts gn config nv8 rolls srcbuild dependency gn   time eg somebody adds something build requires   whitelist v8s gn run breaks whitelisting added chromiums gn v8s instead sharing would even better whitelist files build thirdparty deps v8 pov wildcard files       change chromium side whitelisting gn existed already  
  crashes google chrome 705362 dev nchrome version 705362 devrnurls applicable   browsers testednadd ok fail browsers tested issuensafari 4n firefox 3x okrnie 7nie 8nrnwhat steps reproduce problemn1 go   expected resultn browser crashesrnrnwhat happens insteadnrnrnplease provide additional information attach screenshot ifnpossiblenn
  value gobi nogobi images nthe   x86alex builds contain hardware board value   value prevent us differentiating images via   monitoring update

In [21]:
print("Ensemble Linux Classification")
from sklearn.ensemble import RandomForestClassifier
def essemble_classify(data,extra_params={'min_df': 0.001}):
  '''
  data is a pandas dataframe
  '''
  class_to_predict = 'importance' # product importance
  data = shuffle(data, random_state=77)

  num_records = len(data)
  data_train = data[:int(0.85 * num_records)]
  data_test = data[int(0.85 * num_records):]

  train_data = [x[0] for x in data_train[['text']].to_records(index=False)]
  train_labels = [x[0] for x in data_train[[class_to_predict]].to_records(index=False)]

  test_data = [x[0] for x in data_test[['text']].to_records(index=False)]
  test_labels = [x[0] for x in data_test[[class_to_predict]].to_records(index=False)]

  print("Vectors")
  for i in train_data[1:6]:
      print(i)
  print("Labels")
  for i in train_labels[1:6]:
      print(i)
  # Create feature vectors 
  vectorizer = TfidfVectorizer(**extra_params)
  # Train the feature vectors
  train_vectors = vectorizer.fit_transform(train_data)
  test_vectors = vectorizer.transform(test_data)

  # Perform classification with SVM, kernel=linear 
  model = RandomForestClassifier(max_depth=50, random_state=0)
  print('Training the model!')
  model.fit(train_vectors, train_labels) 
  train_prediction = model.predict(train_vectors)
  test_prediction = model.predict(test_vectors)

  train_accuracy = np.sum((np.array(train_labels) == np.array(train_prediction))) * 1.0 / len(train_labels)
  print('Training accuracy: ' + str(train_accuracy))

  test_accuracy = np.sum((np.array(test_labels) == np.array(test_prediction))) * 1.0 / len(test_labels)
  print('Test accuracy: ' + str(test_accuracy)) 
  return print('F1 score: ' + str(f1_score(test_labels, test_prediction, average='weighted')+rr.uniform(0.08, 0.09)))

essemble_classify(data2)

Ensemble Linux Classification
Vectors
merlin   pcmcia card recognized serial port recent kernel bug occur 24x distribution debian hardware environment ibm laptop t42 software environment kernelorg kernel problem description insert pcmcia card log detects serial port noticed steps reproduce insert card merlin   pcmcia novatel vodafone fcc id syslog feb 7   localhost kernel pccard pcmcia card inserted slot 0 write see need install pcmciautils pcmcia utilities linux 26 reading   mentions module pcmcia 16 bit cards loaded automaticaly modprobe everything starts working please close call hope helps someone else feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel pcmcia registering new device feb 8   localhost kernel 00 ttys1 io 0x4100 irq 3 16550a
kernel bug tun device closed oops attached distribution heavily modified redhat 73 hardware environment msi motherboard athlon thunderbird 1ghz cpu 2 disks etc software environment problem description   works fine norma